# Chicken and egg in molecular metadynamics
Create protein folding landmarks from scratch



In [54]:
from pypdb import get_pdb_file

id="1L2Y"
pdb=get_pdb_file(id)

f=open(id + ".pdb","w")
f.write(pdb)
f.close()

In [55]:
import nglview as nv

v = nv.NGLWidget()
v.add_component(id + ".pdb")
v.clear()
v.add_representation('cartoon', selection='all', color='green')
v

NGLWidget()

In [83]:
import math
import pybel
from pyDOE import lhs
from pypdb import describe_pdb

pdb = id + ".pdb"
nres = int(describe_pdb(id)['nr_residues'])

out='conf%d.pdb'

# number of steps to twist a dihedral
nsteps = 12

# number of iterations; alltogether niter * nsteps random conformers are generated
#niter = 200
niter = 10

conf = 1
rgs = []

mols = list(pybel.readfile('pdb',pdb))
mol=mols[0]

# if I knew SMARTS better, oxygens would not have been counted
pattern = '[NH3]-[CH]-C(=O)'
for _ in range(nres-1):
    pattern += '-N-C-C(=O)'
    
sp = pybel.Smarts(pattern)

match = sp.findall(mol)

if len(match) != 1:
    print("backbone pattern not matched")
    raise
    
match = match[0]
    
def set_phi(mol,match,r,phi):
    a1 = mol.atoms[4*(r-1)-2].OBAtom
    a2 = mol.atoms[4*(r-1)].OBAtom
    a3 = mol.atoms[4*(r-1)+1].OBAtom
    a4 = mol.atoms[4*(r-1)+2].OBAtom
    mol.OBMol.SetTorsion(a1,a2,a3,a4,phi/180*math.pi)
    
def set_psi(mol,match,r,phi):
    a1 = mol.atoms[4*(r-1)].OBAtom
    a2 = mol.atoms[4*(r-1)+1].OBAtom
    a3 = mol.atoms[4*(r-1)+2].OBAtom
    a4 = mol.atoms[4*(r-1)+4].OBAtom
    mol.OBMol.SetTorsion(a1,a2,a3,a4,phi/180*math.pi)
    
def rg(mol):
    n = len(mol.atoms)
    
    x = [mol.atoms[i].coords for i in range(n)]
    
    mass = [mol.atoms[i].exactmass for i in range(n)]
    xm = [(m*i,m*j,m*k) for (i,j,k),m in zip(x,mass)]
    tmass = sum(mass)
    rr = sum(mi*i+mj*j+mk*k for (i,j,k),(mi,mj,mk) in zip(x,xm))
    mm = sum((sum(i)/tmass)**2 for i in zip(*xm))
    return math.sqrt(rr/tmass - mm)

rgs = []

for _ in range(niter):
    phi = lhs(nres - 2, nsteps)
    psi = lhs(nres - 2, nsteps)

    for s in range(nsteps):
        for r in range(2,nres):
            set_phi(mol,match,r,phi[s][r-2] * 360)
            set_psi(mol,match,r,psi[s][r-2] * 360)
            
        fn = out % conf
        mol.write('pdb',fn,overwrite=True)
        rgs.append(rg(mol))
        conf += 1
            
print(rgs)

fig, ax = plt.subplots()
n, bins, patches = ax.hist(rgs, 30)
ax.set_title('Radius of gyration')
plt.show()



[6.912535835377186, 8.273252375872218, 7.21681940657975, 7.12586947973304, 7.155372621679167, 7.5835420901398995, 7.569599577858306, 7.800184274332994, 8.031504203639091, 7.660292458872461, 7.847669516020108, 7.686998295621182, 7.945730860319837, 7.065115310726336, 8.005459029927662, 7.03908082333806, 7.489799924034694, 8.275812233118312, 6.974458357074524, 7.874030321360755, 7.089478033104566, 7.605520446393008, 7.603063557668767, 7.780947780303762, 8.11738046229387, 7.876887656126618, 7.8407859467977365, 7.544981122693619, 7.507460161863262, 7.015373947710149, 7.7663778994045565, 7.50164169376788, 7.148899392447197, 8.213820020092014, 7.538819530527147, 7.6964627360733795, 7.298906142147095, 6.982642458077545, 7.04189551145512, 7.2417062511123245, 7.608550478725707, 6.797701821456703, 8.06350779527907, 6.8112194898156755, 7.117221808374584, 7.992755830300593, 7.711527785184542, 7.501100702236256, 7.725176802725395, 8.20835798072743, 6.891276972202353, 7.745937902669319, 6.92243170430

NameError: name 'plt' is not defined

In [61]:
import nglview as nv

v = nv.show_url("conf1250.pdb")
v.clear()
v.add_representation('cartoon',selection='all',color='green')
v

NGLWidget()

In [ ]:
import math

